# Object Detection Project
taken form multiple sources but mainly from here:
https://towardsdatascience.com/real-time-object-detection-with-tensorflow-detection-model-e7fd20421d5d


you will need to have tensorflow and openCV to run this project
some parts are erroneous as this is taken from a project found online and focused on sample images and not real-time video


Step 1: load packages


In [ ]:
#loading packages
#you will need to have tensorflow and openCV to run this project
#some parts are erroneous as this is taken from a project found online

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


In [ ]:
# This is needed to display the images.
%matplotlib inline

from utils import label_map_util

from utils import visualization_utils as vis_util

Step 2:
Pull in model. For this I am using Tensorflow object detection trained on the coco dataset found here:
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

In [ ]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [ ]:
#TF lingo I don't really understand!

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
#labels from the coco dataset that are pulled from my local as I couldn't get it to work with Github link. You will need to download it to you local and change the file location 

label_map = label_map_util.load_labelmap(r"C:\Users\Brian\models\research\object_detection\data\mscoco_label_map.pbtxt")
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

Step 3: weird matplot stuff for uploading an image. Since we're using real-time video this is unnecessary but I'm leaving it in because I'm lazy

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

Step 4: OpenCV fun
    
    below piece is loading the video capture as being a IP camera I found online. You can also use your local webcam by typing:
       cap = cv2.VideoCapture(0) 
    or a different camera (alternative to laptop webcam with cap = cv2.VideoCapture(1))
        

In [ ]:
import cv2

cap = cv2.VideoCapture('http://93.87.72.254:8090/mjpg/video.mjpg?COUNTER')


Step 5 - building the detection and overlaying on the video

Not gonna lie, a lot of this is mumbo-jumbo to me right now! Basically, it is using the model to detect on the video being run above and overlaying bounding boxes and detected object names from coco

In [ ]:
#building the bounding boxes/assigning detection

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
      ret, image_np = cap.read()
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=4)
#openCV stuff - object detection is the above detection overlayed, and the cv2.resize is resizing the output image
#the last if statement allow you to close it on pressing 'Q'
#imshow is where you can plug in other fun stuff like backround reduction and other OpenCV features!
      cv2.imshow('object detection', cv2.resize(image_np, (1042,586)))
      if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break